In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_pickle('vod.pkl')
df = df[['subsr', 'asset_nm', 'use_tms', 'vod분류']]

# vod 정보 데이터프레임 생성
vod_info = df.groupby(['asset_nm', 'vod분류']).count()[['subsr']].reset_index().rename(columns={'subsr':'subsr_count'})
vod_info['use_tms_sum'] = df.groupby(['asset_nm', 'vod분류']).sum()[['use_tms']].reset_index()['use_tms']
vod_info = vod_info.reset_index().rename(columns={'index':'vod_id'})

# user_log 정보 데이터프레임 생성
user_log = df[['subsr', 'asset_nm', 'use_tms']].merge(vod_info[['vod_id', 'asset_nm']]).sort_values(by='subsr').drop('asset_nm', axis=1)

# 피벗 테이블 만들기 (user별 vod를 시청한 시청시간에 대한 pivot table)
score_matrix = user_log.pivot_table(index='subsr', columns='vod_id', values='use_tms')

# 결측치 제거
score_matrix = score_matrix.fillna(0)

# 비슷한 성향의 user 찾기
import operator
def similar_users(user_id, matrix, k=5):
    
    # 현재 user_id에 대한 데이터프레임 준비
    user = score_matrix[score_matrix.index == user_id]
    
    # 나머지 user들에 대한 정보
    other_user = score_matrix[score_matrix.index != user_id]
    
    # 대상 user와 나머지 user들과의 유사도 계산
    sim = cosine_similarity(user, other_user)[0].tolist()
    
    # 나머지 user들에 대한 목록 생성
    other_users_list = other_user.index.tolist()
    
    # 인덱스/유사도로 이루어진 딕셔너리 생성
    user_sim = dict(zip(other_users_list, sim))
    
    # 딕셔너리 정렬
    user_sim_sorted = sorted(user_sim.items(), key=operator.itemgetter(1))
    
    # 가장 높은 유사도 k개 정렬
    top_users_sim = user_sim_sorted[:k]
    users = [i[0] for i in top_users_sim]
    return users


# vod 추천하기
def recommend_vod(user_index, similar_user_indices, matrix, items=10):
    # 유저와 비슷한 유저 가져오기
    sim_users = matrix[matrix.index.isin(similar_user_indices)]
    
    # 비슷한 유저 평균 계산
    sim_users = sim_users.mean(axis=0)
    sim_users_df = pd.DataFrame(sim_users, columns=['user_similarity'])
    
    # 현재 사용자의 벡터 가져오기
    user_df = score_matrix[score_matrix.index == user_index]
    
    # 현재 사용자의 평가 데이터 정렬
    user_df_transposed = user_df.transpose()
    user_df_transposed.columns = ['time']
    
    # 미시청 vod 목록리스트 만들기
    unseen_vod = user_df_transposed[user_df_transposed['time']==0].index.tolist()
    
    # 미시청 vod 필터링
    sim_users_df_filtered = sim_users_df[sim_users_df.index.isin(unseen_vod)]
    
    # 평균값을 기준으로 내림차순 정렬
    sim_users_df_ordered = sim_users_df_filtered.sort_values(by='user_similarity', ascending=False)
    
    # 상위 n개 값 가져오기
    top_n_vod = sim_users_df_ordered.head(items)
    top_n_vod_indices = top_n_vod.index.tolist()
    
    # vod_info 데이터프레임에서 top_n값 찾기
    vod_information = vod_info[vod_info['vod_id'].isin(top_n_vod_indices)]
    vod_information
    
    return vod_information


recommend_vod(59930000, similar_users(59930000, score_matrix), score_matrix)

,vod_id,asset_nm,vod분류,subsr_count,use_tms_sum
68,68,(HD)그것이알고싶다 1367회(23/09/02),TV 시사/교양 > 기타,6,16882
69,69,(HD)그것이알고싶다 1368회(23/09/09),TV 시사/교양 > 기타,5,14613
70,70,(HD)그것이알고싶다 1369회(23/09/16),TV 시사/교양 > 기타,5,16342
132,132,(HD)런닝맨 666회(23/08/06),TV 연예/오락 > 기타,3,6997
133,133,(HD)런닝맨 667회(23/08/13),TV 연예/오락 > 기타,10,23652
134,134,(HD)런닝맨 668회(23/08/20),TV 연예/오락 > 기타,11,26679
135,135,(HD)런닝맨 669회(23/08/27),TV 연예/오락 > 기타,11,8901
136,136,(HD)런닝맨 670회(23/09/03),TV 연예/오락 > 기타,8,20745
738,738,금이야 옥이야 121회(23/09/15),TV드라마 > 기타,2,3338
2291,2291,후궁-제왕의첩,영화 > 멜로,30,25098
